In [ ]:
import csv
import json
import os
import datetime
from collections import defaultdict

# --- Configuration ---
PATIENTS_FILE = 'patients.csv'
BACKUP_FILE = 'backup_records.json'
LOG_FILE = 'audit.log'

class HospitalSystem:
    def __init__(self):
        self.patients = {}  # {id: {'name': str, 'diagnosis': str, 'medications': list}}
        self.appointments = []  # List of tuples: (date, time, doctor, patient_id)
        self.load_patients_csv()

    # --- File Handling ---

    def log_action(self, action_type, details):
        """
        Appends actions to audit.log in a strictly parseable format for rollback.
        Format: ACTION_TYPE|JSON_DETAILS
        """
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # storing details as a JSON string within the log line for easy parsing later
        log_entry = f"{timestamp}|{action_type}|{json.dumps(details)}\n"
        
        with open(LOG_FILE, 'a') as f:
            f.write(log_entry)

    def load_patients_csv(self):
        """Loads patient data from CSV at startup."""
        if not os.path.exists(PATIENTS_FILE):
            print(f"Warning: {PATIENTS_FILE} not found. Starting with empty records.")
            return

        try:
            with open(PATIENTS_FILE, mode='r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    pid = int(row['id'])
                    # Convert medications string "med1,med2" to list
                    meds = row['medications'].split(',') if row['medications'] else []
                    self.patients[pid] = {
                        'name': row['name'],
                        'diagnosis': row['diagnosis'],
                        'medications': meds
                    }
            print(f"Loaded {len(self.patients)} patients from CSV.")
        except Exception as e:
            print(f"Error loading CSV: {e}")

    def backup_records(self):
        """Backs up current state to JSON."""
        data = {
            'patients': self.patients,
            'appointments': self.appointments
        }
        try:
            with open(BACKUP_FILE, 'w') as f:
                json.dump(data, f, indent=4)
            print("Backup successful.")
            self.log_action("BACKUP", "System state backed up")
        except Exception as e:
            print(f"Backup failed: {e}")

    def recover_backup(self):
        """Recovers from JSON backup if CSV is corrupt/missing."""
        try:
            with open(BACKUP_FILE, 'r') as f:
                data = json.load(f)
                # JSON keys are always strings, convert patient IDs back to ints
                self.patients = {int(k): v for k, v in data.get('patients', {}).items()}
                # Convert appointment lists back to tuples
                self.appointments = [tuple(appt) for appt in data.get('appointments', [])]
            print("Recovery from backup successful.")
        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Critical: Backup corrupt or missing. {e}")

    # --- Core Functionality ---

    def add_patient(self, pid, name, diagnosis, medications):
        if pid in self.patients:
            print(f"Error: Patient ID {pid} already exists.")
            return

        self.patients[pid] = {'name': name, 'diagnosis': diagnosis, 'medications': medications}
        print(f"Patient {name} added.")
        self.log_action("ADD_PATIENT", {'id': pid, 'name': name, 'diagnosis': diagnosis, 'medications': medications})

    def schedule_appointment(self, date_str, time_str, doctor, patient_id):
        # 1. Validation: Patient ID
        if patient_id not in self.patients:
            print(f"Error: Patient ID {patient_id} not found.")
            return

        # 2. Validation: Time Overlaps
        new_appt = (date_str, time_str, doctor, patient_id)
        for appt in self.appointments:
            # Check if same doctor has an appointment at same date/time
            if appt[0] == date_str and appt[1] == time_str and appt[2] == doctor:
                print(f"Error: Doctor {doctor} is busy at {time_str} on {date_str}.")
                return

        self.appointments.append(new_appt)
        print(f"Appointment scheduled for Patient {patient_id}.")
        
        # Log specifically for rollback capability
        self.log_action("SCHEDULE", {'date': date_str, 'time': time_str, 'doctor': doctor, 'patient_id': patient_id})

    def cancel_appointment(self, date_str, time_str, doctor, patient_id):
        target = (date_str, time_str, doctor, patient_id)
        if target in self.appointments:
            self.appointments.remove(target)
            print("Appointment canceled.")
            # We log the full details so we can re-add it during a rollback
            self.log_action("CANCEL", {'date': date_str, 'time': time_str, 'doctor': doctor, 'patient_id': patient_id})
        else:
            print("Error: Appointment not found.")

    def generate_treatment_report(self):
        """Groups patients by diagnosis."""
        report = defaultdict(list)
        for pid, info in self.patients.items():
            report[info['diagnosis']].append(info['name'])
        
        print("\n--- Treatment Report ---")
        for diagnosis, names in report.items():
            print(f"Diagnosis: {diagnosis} | Patients: {', '.join(names)}")
        print("------------------------\n")

    # --- The Hard Challenge: Rollback ---

    def rollback(self, steps=1):
        """
        Undoes the last N actions by parsing audit.log from the bottom up.
        Max limit: 3 actions.
        """
        if steps > 3:
            print("Limit exceeded: Can only rollback last 3 actions.")
            steps = 3

        if not os.path.exists(LOG_FILE):
            print("No log file found to rollback.")
            return

        # Read all lines
        with open(LOG_FILE, 'r') as f:
            lines = f.readlines()

        if not lines:
            print("Log is empty.")
            return

        # Process the last N lines in reverse
        actions_to_undo = lines[-steps:]
        # We also need to truncate the log file to remove these undone actions
        remaining_lines = lines[:-steps]

        print(f"\nRolling back {len(actions_to_undo)} action(s)...")

        # Iterate backwards through the lines we want to undo
        for line in reversed(actions_to_undo):
            try:
                parts = line.strip().split('|')
                if len(parts) < 3: continue
                
                # parts[0] is timestamp, parts[1] is ACTION, parts[2] is JSON details
                action_type = parts[1]
                details = json.loads(parts[2])

                if action_type == "SCHEDULE":
                    # Inverse: Remove the appointment
                    # We access raw list to avoid re-logging the cancel action
                    target = (details['date'], details['time'], details['doctor'], details['patient_id'])
                    if target in self.appointments:
                        self.appointments.remove(target)
                        print(f"Undid SCHEDULE: Removed appt for {details['patient_id']}")

                elif action_type == "CANCEL":
                    # Inverse: Add the appointment back
                    target = (details['date'], details['time'], details['doctor'], details['patient_id'])
                    self.appointments.append(target)
                    print(f"Undid CANCEL: Restored appt for {details['patient_id']}")

                elif action_type == "ADD_PATIENT":
                    # Inverse: Delete the patient
                    pid = details['id']
                    if pid in self.patients:
                        del self.patients[pid]
                        print(f"Undid ADD_PATIENT: Removed patient {pid}")

                else:
                    print(f"Skipping non-reversible action: {action_type}")

            except json.JSONDecodeError:
                print("Error parsing log line. Skipping.")

        # Rewrite log file without the undone actions
        with open(LOG_FILE, 'w') as f:
            f.writelines(remaining_lines)
        print("Rollback complete.\n")


# --- Usage Example ---
if __name__ == "__main__":
    # Create dummy CSV for testing if it doesn't exist
    if not os.path.exists(PATIENTS_FILE):
        with open(PATIENTS_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['id', 'name', 'diagnosis', 'medications'])
            writer.writerow([101, 'John Doe', 'Flu', 'Tamiflu,Ibuprofen'])
            writer.writerow([102, 'Jane Smith', 'Fracture', 'Painkiller'])

    system = HospitalSystem()

    # 1. Add a new patient
    system.add_patient(103, "Alice Brown", "Flu", ["Water", "Rest"])

    # 2. Schedule Appointments
    system.schedule_appointment("2023-10-27", "10:00", "Dr. House", 101)
    system.schedule_appointment("2023-10-27", "11:00", "Dr. House", 103)
    
    # 3. Validation Fail (Overlap)
    system.schedule_appointment("2023-10-27", "10:00", "Dr. House", 102)

    # 4. Cancel Appointment
    system.cancel_appointment("2023-10-27", "10:00", "Dr. House", 101)

    # 5. Generate Report
    system.generate_treatment_report()

    # 6. Backup
    system.backup_records()

    # 7. ROLLBACK TEST (Undo the Backup, Cancel, and Schedule)
    # Note: 'Backup' action in my logic is skipped as non-reversible, 
    # so it will effectively undo the Cancel and the Schedule.
    system.rollback(steps=3) 
    
    # Check status after rollback
    print(f"Current Appointments: {system.appointments}")